In [6]:
import os
import csv
import pandas as pd
import numpy as np

import datetime
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import squarify

from sklearn import model_selection, preprocessing, metrics
plt.style.use('fivethirtyeight')

print(os.getcwd())
print(os.listdir("../"))

/Users/vishakhadudani/APR
['Untitled Folder', '.Directory-1534269790', 'Music', '.condarc', 'Untitled1.ipynb', '.DS_Store', 'nltk_data', 'settings.cfg', '.CFUserTextEncoding', 'midterm-2-takehome.pdf', 'Untitled.ipynb', "NLP-Paper - CNN'.pdf", '.local', 'Creative Cloud Files', 'Pictures', 'sampleTensorFlow', 'Test_tensor.ipynb', '2008.json', '.ipython', 'Desktop', 'Library', '.matplotlib', 'test.py', '.oracle_jre_usage', 'MS SBU', 'Test-Notebook.ipynb', '.bash_sessions', 'Word_cloud_nlp.png', 'untitled.txt', '.bash_profile-anaconda2.bak', 'Public', 'NLP2014Paper.pdf', 'examples', 'vector_plit.png', '.anaconda', 'BF00367750.pdf', 'APR', 'Movies', 'Applications', '.Trash', '.ipynb_checkpoints', '.jupyter', '.keras', 'helloworld.py', 'Documents', 'venv', 'nyc-taxi-fare-prediction.ipynb', 'mag_2018.csv', '.bash_profile', 'data', 'Downloads', '.python_history', 'notebook.tex', '.gitconfig', 'CSE538TensorflowTutorial.ipynb', '.bash_history', 'anaconda2', '.conda']


In [7]:
df = pd.read_csv("../mag_2018.csv")

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1211000 entries, 0 to 1210999
Data columns (total 16 columns):
Unnamed: 0     1211000 non-null int64
AA             1211000 non-null object
C              50789 non-null object
CC             1211000 non-null int64
D              1211000 non-null object
ECC            1211000 non-null int64
F              1197152 non-null object
Id             1211000 non-null int64
J              1019164 non-null object
RId            835626 non-null object
Ti             1211000 non-null object
VFN            1147111 non-null object
W              1210997 non-null object
Y              1211000 non-null int64
kesEntityId    1211000 non-null int64
logprob        1211000 non-null float64
dtypes: float64(1), int64(6), object(9)
memory usage: 147.8+ MB


In [9]:
def get_block_avg_usage(df):
    for dtype in ['float','int','object']:
        selected_dtype = df.select_dtypes(include=[dtype])
        mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
        mean_usage_mb = mean_usage_b / 1024 ** 2
        print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))
        del mean_usage_b, mean_usage_mb, selected_dtype

In [10]:
df.head(5)

,Unnamed: 0,AA,C,CC,D,ECC,F,Id,J,RId,Ti,VFN,W,Y,kesEntityId,logprob
0,0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064
1,1,"[{'S': 1, 'AuId': 2618360078, 'AfN': 'technica...",NaN,1787,2018-02-23,1787,"[{'FId': 2779853757, 'FN': 'penicillic acid'},...",2789134836,"{'JN': 'world mycotoxin journal', 'JId': 70096...","[2073569767, 1989318771, 2109782899, 214495155...",a critical review of producers of small lacton...,World Mycotoxin Journal,"['acid', 'critical', 'lactone', 'moniliformin'...",2018,1349261,-16.192
2,2,"[{'AuId': 2498139363, 'S': 1, 'AuN': 'r r m pa...",NaN,1785,2018-03-01,1785,"[{'FId': 2779550045, 'FN': 'patulin'}, {'FId':...",2786183769,"{'JN': 'int j food microbiol', 'JId': 132626406}",NaN,alternative patulin pathway unproven,International Journal of Food Microbiology,"['alternative', 'pathway', 'patulin', 'unproven']",2018,1350611,-16.193
3,3,"[{'S': 1, 'AuId': 2127898042, 'AfN': 'google',...",NaN,1957,2018-04-01,1957,"[{'FId': 65885262, 'FN': 'scale space segmenta...",2412782625,"{'JN': 'pami', 'JId': 199944782}","[1686810756, 2618530766, 2194775991, 209711776...",deeplab semantic image segmentation with deep ...,IEEE Transactions on Pattern Analysis and Mach...,"['atrous', 'connected', 'convolution', 'convol...",2018,1635532,-16.388
4,4,"[{'S': 1, 'AuId': 2761591088, 'AfN': 'leiden u...",NaN,293,2018-08-01,293,"[{'FId': 2778316820, 'FN': 'photometric system...",2798336535,"{'JN': 'astron astrophys', 'JId': 205231332}","[2011301426, 2142220522, 2061939373, 205293750...",gaia data release 2 summary of the contents an...,Astronomy and Astrophysics,"['2', 'contents', 'data', 'gaia', 'properties'...",2018,1744805,-16.455


In [11]:
# Make more sensible column names
rename_dict = {"AA": "Authors", 
               "C": "Conference", 
               "CC": "Citation_count",
               "ECC": "Expected_count",
               "D": "Date", 
               "F": "Domain", 
               "J": "Journal", 
               "RId": "References", 
               "Ti": "Title", 
               "VFN": "Venue", 
               "W": "Words", 
               "Y": "Year", 
               "logprob":"Popularity",
               "Unnamed: 0": "Row_num"}
df.rename(index=str, columns=rename_dict, inplace=True)

In [12]:
df.select_dtypes(include="number").describe()

,Row_num,Citation_count,Expected_count,Id,Year,kesEntityId,Popularity
count,1.211000e+06,1.211000e+06,1.211000e+06,1.211000e+06,1211000.0,1.211000e+06,1.211000e+06
mean,6.054995e+05,1.064620e+00,1.064997e+00,2.797143e+09,2018.0,2.260905e+08,-2.118847e+01
std,3.495857e+05,1.755177e+01,1.755261e+01,1.486480e+08,0.0,9.040128e+07,6.017534e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,4.727120e+05,2018.0,1.190005e+06,-2.200200e+01
25%,3.027498e+05,0.000000e+00,0.000000e+00,2.785024e+09,2018.0,1.142518e+08,-2.167500e+01
50%,6.054995e+05,0.000000e+00,0.000000e+00,2.799726e+09,2018.0,2.779343e+08,-2.129900e+01
75%,9.082492e+05,1.000000e+00,1.000000e+00,2.883550e+09,2018.0,2.864866e+08,-2.081000e+01
max,1.210999e+06,6.651000e+03,6.651000e+03,2.898614e+09,2018.0,2.941568e+08,-1.606400e+01


In [13]:
df.select_dtypes(exclude="number").describe()

,Authors,Conference,Date,Domain,Journal,References,Title,Venue,Words
count,1211000,50789,1211000,1197152,1019164,835626,1211000,1147111,1210997
unique,1145040,1256,363,1105872,18787,828087,1208460,42772,1207408
top,"[{'S': 1, 'AuId': 2617818522, 'AfN': 'beckman ...","{'CN': 'cleo', 'CId': 2623670637}",2018-01-01,"[{'FId': 554144382, 'FN': 'performance art'}, ...","{'JN': 'plos one', 'JId': 202381698}","[2028520323, 1657433554, 2782514705, 273981532...",open access is there a predator at the door,PLOS ONE,"['access', 'door', 'open', 'predator']"
freq,3810,1643,182147,4454,14841,1206,11,14841,11


In [115]:
df[df.duplicated(["Title"])].sort_values(by=["Title"])

,Row_num,Authors,Conference,Citation_count,Date,ECC,Domain,Id,Journal,References,Title,Venue,Words,Year,kesEntityId,Popularity,month


In [14]:
# df 
df.drop_duplicates(subset=['Title'], inplace=True)

In [15]:
df.isna().sum()

Row_num                 0
Authors                 0
Conference        1157829
Citation_count          0
Date                    0
Expected_count          0
Domain              13793
Id                      0
Journal            191087
References         373689
Title                   0
Venue               63804
Words                   3
Year                    0
kesEntityId             0
Popularity              0
dtype: int64

In [16]:
df["Conference"].value_counts(dropna=False)

NaN                                         1157829
{'CN': 'cleo', 'CId': 2623670637}              1626
{'CN': 'chi', 'CId': 1163450153}               1467
{'CN': 'icra', 'CId': 1163902177}              1259
{'CN': 'icassp', 'CId': 1121227772}            1137
{'CN': 'icc', 'CId': 1130451194}               1026
{'CN': 'cvpr', 'CId': 1158167855}              1003
{'CN': 'aaai', 'CId': 1184914352}               872
{'CN': 'nips', 'CId': 1127325140}               843
{'CN': 'acc', 'CId': 1190039108}                792
{'CN': 'ijcai', 'CId': 1203999783}              626
{'CN': 'icml', 'CId': 1180662882}               622
{'CN': 'iscas', 'CId': 1192710900}              543
{'CN': 'icse', 'CId': 1174403976}               427
{'CN': 'ccc', 'CId': 2623792093}                414
{'CN': 'interspeech', 'CId': 1177287137}        411
{'CN': 'ofc', 'CId': 2621451516}                404
{'CN': 'naacl', 'CId': 1173951661}              394
{'CN': 'eccv', 'CId': 1124077590}               385
{'CN': 'icip

In [17]:
df.iloc[0:1,-1]

0   -16.064
Name: Popularity, dtype: float64

In [18]:
df["kesEntityId"].is_unique

True

In [19]:
df["Id"].is_unique

True

In [20]:
# expected citation count is almost same as citation count
df[df["Citation_count"] != (df["ECC"])]

KeyError: 'ECC'

In [21]:
# create month field
df["Month"] = pd.to_datetime(df["Date"]).dt.month

In [22]:
df['Month'].head(5)

0    1
1    2
2    3
3    4
4    8
Name: Month, dtype: int64

In [27]:
def clean_data(df):
    #removing kesEntityId as simillar fields are already present
    df.drop(columns=["Row_num","kesEntityId"],inplace=True)
    df["Popularity"] = df["Popularity"].apply(pd.to_numeric,downcast='float')
    df["Year"] = df["Year"].apply(pd.to_numeric,downcast='unsigned')
    df["Month"] = df["Month"].apply(pd.to_numeric,downcast='unsigned')
    

In [25]:
def clean_data(df):
    #since first column contains row number
    df.drop(df.columns[0], axis=1, inplace=True)
    #removing  and kesEntityId as simillar fields are already present
    df.drop(columns=["Row_num","ECC","kesEntityId"])
    df["Popularity"] = df["Popularity"].apply(pd.to_numeric,downcast='float')
    df["Year"] = df["Year"].apply(pd.to_numeric,downcast='unsigned')
    df = df.dropna(subset=['Venue'])

In [28]:
clean_data(df)

KeyError: "['Row_num'] not found in axis"

In [29]:
df_pub = pd.read_csv("Data/ProcessedScopusData.csv")

In [30]:
def append_pub_score(df):
    print("Unique Publications from Scopus:",len(df_pub.Title.unique()))
    # Analysing Common Publications
    vfn_list = df.Venue.unique().tolist()
    scopus_list = df_pub.Title.unique().tolist()
    count = 0
    for pub in scopus_list:
        if pub in vfn_list:
            count +=1
    print("Publications present in Dataset:",count)
    print("Shape Before",df.shape)
    df = df[df.Venue.isin(scopus_list)]
    print("Shape After",df.shape)
    #score appending
    score_dict = pd.Series(df_pub.SJR.values,index=df_pub.Title).to_dict()
    df["Publication_Rank"] = df.apply(lambda row: score_dict[row["VFN"]],axis = 1)
    return df

def extract_field(row):
    val = row["Domain"]
    index = val.rfind("FN")
    val = val[index+3:len(val)]
    val = val.strip(":}] '")
    return val

def get_num_authors(row):
    vals = row["Authors"].split("},")
    return len(vals)

In [31]:
df = append_pub_score(df)

Unique Publications from Scopus: 1231
Publications present in Dataset: 904
Shape Before (1208460, 16)
Shape After (143303, 16)


KeyError: ('VFN', 'occurred at index 8')

In [32]:
label_dict = {"C":0,"J":1,"CJ":2,"O":3}
def find_conference_type(row):
    if pd.isnull(row["Conference"]) and pd.isnull(row["Journal"]):
        return label_dict["O"]
    elif pd.isnull(row["Conference"]):
        return label_dict["J"]
    elif pd.isnull(row["Journal"]):
        return label_dict["C"]
    else:
        return label_dict["CJ"]

In [33]:
df["Publication Type"] = df.apply(lambda row:find_conference_type(row),axis=1)
df["Number_Of_authors"] = df.apply(lambda row:get_num_authors(row), axis = 1)

In [34]:
df.shape

(1208460, 18)

In [35]:
def tidy_split(df, column, sep='|', keep=False):
    indexes = list()
    id_values = list()
    name_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            id_values.append(presplit)
        for value in values:
            indexes.append(i)
            val = value
            index1 = val.find("AuId")
            index2 = val.find("AfN")
            index3 = val.find("'S'")
            if index3 < index1:
                val = val[index1+4:index2]
                val = val.strip("', :")
            else:
                val = val[index1+4:index3]
                val = val.strip("', :")                
            id_values.append(val)
            index1 = value.find("AuN")
            val = value[index1+3:]
            val = val.strip("': }]")
            name_values.append(val)
    new_df = df.iloc[indexes, :].copy()
    new_df[column+"_Id"] = id_values
    new_new_df = new_df.iloc[:, :].copy()
    new_new_df[column+"_Name"] = name_values
    return new_new_df

In [36]:
df_big = tidy_split(df,"Authors","},")

In [37]:
df_big.head(10)

,Authors,Conference,Citation_count,Date,Expected_count,Domain,Id,Journal,References,Title,Venue,Words,Year,kesEntityId,Popularity,Month,Publication Type,Number_Of_authors,Authors_Id,Authors_Name
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2592382599,lubing bai
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2558648277,ze wang
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2751351928,yamin han
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2754888086,zongyan zuo
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2486643139,bin liu
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2416985679,mengna yu
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2585848087,haijuan zhang
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2609093794,jinyi lin
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 2011472471, 2729523528, 231340543...",diarylfluorene based nano molecules as dopant ...,Nano Energy,"['cells', 'diarylfluorene', 'dopant', 'flexibl...",2018,1190005,-16.064,1,1,16,2586613702,yingdong xia
0,"[{'S': 1, 'AuId': 2592382599, 'AfN': 'center f...",NaN,1872,2018-01-01,1872,"[{'FId': 191952053, 'FN': 'dopant'}, {'FId': 1...",2782514705,"{'JN': 'nano energy', 'JId': 2497301991}","[2057327293, 

In [38]:
df_big.shape

(7536329, 20)

In [42]:
adj_list = {}
for key,row in df_big.iterrows():
    curr_list = adj_list.get(row["Id"], [])
    curr_list.extend((map(int,row["References"].strip(",").split(","))))
    curr_list = list(set(curr_list))
    adj_list[key] = curr_list
adj_list

ValueError: invalid literal for int() with base 10: '[2057327293'